In [1]:
import os
os.environ['HF_HOME'] = '/home/ec2-user/SageMaker/HF_HOME'
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3"

import torch
import datasets
from transformers import DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer

!echo $HF_HOME


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so
CUDA SETUP: CUDA runtime path found: /home/ec2-user/anaconda3/envs/pytorch_p310/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118_nocublaslt.so...


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: Compute capability < 7.5 detected! Only slow 8-bit matmul is supported for your GPU!
  warn(msg)


/home/ec2-user/SageMaker/HF_HOME


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_ckpt='gogamza/kobart-base-v2'
# device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt) #.to('cuda:0')

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
special_tokens_dict = {'additional_special_tokens':['<hl>']}
tokenizer.add_special_tokens(special_tokens_dict)

model.resize_token_embeddings(len(tokenizer))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


Embedding(30001, 768)

In [3]:
IGNORE_PAD_TOKEN_FOR_LOSS = True
PADDING="max_length"

In [4]:
dataset_korquad = datasets.load_dataset('lmqg/qg_koquad')

Found cached dataset qg_koquad (/home/ec2-user/SageMaker/HF_HOME/datasets/lmqg___qg_koquad/qg_koquad/5.0.1/a6f4a2e67e072b2d20dfa43c30cebc1cb871e8fea6efeeb41e69a630ec59f103)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
# def preprocess_function(examples):
#     inputs = [ex[source_lang] for ex in examples["translation"]]
#     targets = [ex[target_lang] for ex in examples["translation"]]
#     # inputs = [prefix + inp for inp in inputs]
#     model_inputs = tokenizer(inputs, max_length=data_args.max_source_length, padding=padding, truncation=True)

#     # Tokenize targets with the `text_target` keyword argument
#     labels = tokenizer(text_target=targets, max_length=max_target_length, padding=padding, truncation=True)

#     # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
#     # padding in the loss.
#     if padding == "max_length" and data_args.ignore_pad_token_for_loss:
#         labels["input_ids"] = [
#             [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
#         ]

#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

In [6]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch["paragraph_answer"], max_length=1024, padding=PADDING, truncation=True)
    target_encodings = tokenizer(example_batch["question"], max_length=1024, padding=PADDING, truncation=True)
    
    if PADDING == "max_length" and IGNORE_PAD_TOKEN_FOR_LOSS:
        target_encodings["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in target_encodings["input_ids"]
        ]
    
    return {"input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]}

dataset_korquad_pt = dataset_korquad.map(convert_examples_to_features, batched=True)

columns = ["input_ids", "labels", "attention_mask"]
dataset_korquad_pt.set_format(type="torch", columns=columns)

Loading cached processed dataset at /home/ec2-user/SageMaker/HF_HOME/datasets/lmqg___qg_koquad/qg_koquad/5.0.1/a6f4a2e67e072b2d20dfa43c30cebc1cb871e8fea6efeeb41e69a630ec59f103/cache-c1bea79d3ffc8415.arrow
Loading cached processed dataset at /home/ec2-user/SageMaker/HF_HOME/datasets/lmqg___qg_koquad/qg_koquad/5.0.1/a6f4a2e67e072b2d20dfa43c30cebc1cb871e8fea6efeeb41e69a630ec59f103/cache-a76bee0018d6541a.arrow
Loading cached processed dataset at /home/ec2-user/SageMaker/HF_HOME/datasets/lmqg___qg_koquad/qg_koquad/5.0.1/a6f4a2e67e072b2d20dfa43c30cebc1cb871e8fea6efeeb41e69a630ec59f103/cache-5d8e965861c86160.arrow


In [7]:
training_args = TrainingArguments(
    output_dir='QG_kobart_korquad',
    num_train_epochs=5,
    warmup_steps=100,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    logging_steps=10,
    push_to_hub=False,
    evaluation_strategy='steps',
    eval_steps=200,
    save_steps=200,
    gradient_accumulation_steps=8,
    fp16=True,
    save_total_limit=4,
)

In [8]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model, 
    args=training_args,
    tokenizer=tokenizer, data_collator=seq2seq_data_collator,
    train_dataset=dataset_korquad_pt["train"], 
    eval_dataset=dataset_korquad_pt["validation"]
)

In [ ]:
%%time
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: daydrill. Use `wandb login --relogin` to force relogin


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss
200,1.723800,1.684276
400,1.604600,1.584406


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [10]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [11]:
dataset_korquad['test'][0]

{'answer': '남부군',
 'paragraph_question': "question: 임창정이 1990년 단역으로 영화배우 첫 데뷔하게 된 영화는?, context: 본관은 풍천이며, 경기도 이천에서 태어났다. 1990년 영화 《남부군》에서 단역으로 영화배우 첫 데뷔에 이어 같은 해 KBS 드라마 《지구인》에서 단역으로 출연하였고 이듬해 MBC 《여명의 눈동자》를 통해 단역으로 출연하였다. 이후 다수의 영화에 단역 및 조역으로 출연하다가 1995년 《거짓같은 진실》을 통해서 가수로 데뷔했으나 노래를 홍보하기 위해 나갔던 라디오에서 이 노래보단, '이미 나에게로'가 좋다고 하고, 본인도 그렇게 느껴 타이틀곡을 <이미 나에게로>로 변경하였다. 1997년 영화 《Beat》의 환규 역으로 출연하였고 그 해 3집 《그때 또 다시》를 발매하여 일약 스타덤에 올랐다. 3집 앨범으로 가요톱10 5주연속 골든컵, 1997년 KBS 가요대상 을 수상하였고 영화 《Beat》로 1997년 대종상영화제 남우조연상과 1998년 백상예술대상 신인연기상을 수상하였다. 1998년 4집 《별이 되어》를 발매하고 활동하며 첫 주연 영화 《엑스트라》가 개봉되었고 이후 꾸준히 앨범과 영화를 병행하며 활동하다 2003년 10집 《Bye》를 마지막으로 영화전념을 위해 가수분야에서는 잠정 은퇴선언하였다.",
 'question': '임창정이 1990년 단역으로 영화배우 첫 데뷔하게 된 영화는?',
 'sentence': '1990년 영화 《 남부군 》에서 단역으로 영화배우 첫 데뷔에 이어 같은 해 KBS 드라마 《지구인》에서 단역으로 출연하였고 이듬해 MBC 《여명의 눈동자》를 통해 단역으로 출연하였다.',
 'paragraph': "본관은 풍천이며, 경기도 이천에서 태어났다. 1990년 영화 《남부군》에서 단역으로 영화배우 첫 데뷔에 이어 같은 해 KBS 드라마 《지구인》에서 단역으로 출연하였고 이듬해 MBC 《여명의 눈동자》를 통해 단역으로 출연하였다. 이후 다수의 영화에 단역 및 조역으로 

In [12]:
dataset_korquad['test'][0]['paragraph_answer']

"본관은 풍천이며, 경기도 이천에서 태어났다. 1990년 영화 《<hl> 남부군 <hl>》에서 단역으로 영화배우 첫 데뷔에 이어 같은 해 KBS 드라마 《지구인》에서 단역으로 출연하였고 이듬해 MBC 《여명의 눈동자》를 통해 단역으로 출연하였다. 이후 다수의 영화에 단역 및 조역으로 출연하다가 1995년 《거짓같은 진실》을 통해서 가수로 데뷔했으나 노래를 홍보하기 위해 나갔던 라디오에서 이 노래보단, '이미 나에게로'가 좋다고 하고, 본인도 그렇게 느껴 타이틀곡을 <이미 나에게로>로 변경하였다. 1997년 영화 《Beat》의 환규 역으로 출연하였고 그 해 3집 《그때 또 다시》를 발매하여 일약 스타덤에 올랐다. 3집 앨범으로 가요톱10 5주연속 골든컵, 1997년 KBS 가요대상 을 수상하였고 영화 《Beat》로 1997년 대종상영화제 남우조연상과 1998년 백상예술대상 신인연기상을 수상하였다. 1998년 4집 《별이 되어》를 발매하고 활동하며 첫 주연 영화 《엑스트라》가 개봉되었고 이후 꾸준히 앨범과 영화를 병행하며 활동하다 2003년 10집 《Bye》를 마지막으로 영화전념을 위해 가수분야에서는 잠정 은퇴선언하였다."

In [22]:
_input = dataset_korquad['test'][2]['paragraph_answer']
_input

'한반도의 늑대는 남한의 경우 1980년대를 마지막으로 사라졌다. 그리고 동물원 사육개체도 1997년을 마지막으로 죽어버렸다. 늑대가 사라지면서 현재 그 먹이가 되는 고라니나 노루, 멧돼지와 너구리등 일부 야생동물들이 개체수가 너무 늘어나면서 농작물을 먹어 치워 여러가지 문제가 발생하였다. 그러자, 일부 동물 보호단체에서 중국에서 만주산 늑대 4마리를 들여와 재도입계획을 세우고 있다. 또한 민간단체에서도 몽골에서 늑대를 들여와 복원을 추진하고 있으며 <hl> 환경부 <hl>에서는 북한과 연해주에서 늑대를 도입할 계획이다. 하지만 한반도의 늑대 복원계획에 대해 문제가 있다. 대륙에서 들여올 늑대가 한반도 아종이 아니라 다른 아종이기 때문에 외래종의 늑대 도입이 될 가능성이 있다. 현재 늑대가 복원이 되면 증가하여 여러 문제가 있는 멧돼지나 고라니등 번식력이 좋은 우제류등 야생동물 개체수를 조절이 가능하다. 하지만 역시 늑대가 사람을 해칠 가능성 역시 제기되고 있다.'

In [23]:
_input = tokenizer(_input, return_tensors='pt', return_token_type_ids=False, max_length=1024)

In [24]:
_input['input_ids'] = _input['input_ids'].to('cuda')
_input['attention_mask'] = _input['attention_mask'].to('cuda')

In [32]:
gened = model.generate(**_input, 
    max_new_tokens=100,
    early_stopping=True,
    do_sample=True,
    eos_token_id=2,
)

In [33]:
gened_q = tokenizer.decode(gened[0]) #.split('### 답변:')[-1].strip()

In [34]:
gened_q

'</s> 늑대를 복원한 몽골의 연해주에 있는 정부 부처는? 대한민국은? 중국으로? 북한에서 연해주에서 늑대를 도입하고자 계획이 나온 정부 부처는? 어디인가????????? 우리나라 민간단체에서 늑대를 도입하려는 기관은????????????????????????????????????????????</s>'